In [1]:
!pip install --upgrade tables
!pip install --upgrade eli5
!pip install --upgrade xgboost
!pip install --upgrade hyperopt


     |████████████████████████████████| 4.3MB 2.7MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.8MB/s 
     |████████████████████████████████| 109.8MB 93kB/s 
  Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90
     |████████████████████████████████| 1.9MB 2.8MB/s 
     |████████████████████████████████| 1.7MB 43.5MB/s 
  Created wheel for networkx: filename=networkx-2.2-py2.py3-none-any.whl size=1527322 sha256=05cf642fbd6306cf41ae029d1ef03b6df42312c80f994ddcbcb1ab523e8897ed
  Stored in directory: /root/.cache/pip/wheels/68/f8/29/b53346a112a07d30a5a84d53f19aeadaa1a474897c0423af91
Successfully built networkx
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: networkx 2.4
    Uninstalling networkx-2.4

In [2]:
import pandas as pd
import numpy as np
#from sklearn.dummy import DummyRegressor
#from sklearn.tree import DecisionTreeRegressor
#from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

from hyperopt import hp,fmin,tpe,STATUS_OK

import eli5
from eli5.sklearn import PermutationImportance

#import matplotlib.pyplot as plt
#import seaborn as sns

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [4]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/dw_matrix_two/dw_matrix_car


In [5]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [0]:
SUFFIX_CAT='__cat'
for feat in df.columns:
  if isinstance(df[feat][0],list): continue

  factorized_value=df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_value 
  else:
    df[feat + SUFFIX_CAT] = factorized_value

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x)=='None' else int(x))
df['param_moc']=df['param_moc'].map(lambda x: -1 if str(x)=='None' else int(x.split(' ')[0]))
df['param_pojemność-skokowa']=df['param_pojemność-skokowa'].map(lambda x: -1 if str(x)=='None' else int(str(x).split('cm')[0].replace(' ','')))



In [15]:
feats = [ 'param_napęd__cat','param_rok-produkcji','param_stan__cat','param_skrzynia-biegów__cat','param_faktura-vat__cat','param_moc','param_marka-pojazdu__cat','feature_kamera-cofania__cat','param_typ__cat','param_pojemność-skokowa','seller_name__cat','feature_wspomaganie-kierownicy__cat','param_model-pojazdu__cat','param_wersja__cat','param_kod-silnika__cat','feature_system-start-stop__cat','feature_asystent-pasa-ruchu__cat','feature_czujniki-parkowania-przednie__cat','feature_łopatki-zmiany-biegów__cat','feature_regulowane-zawieszenie__cat' ]

xgb_params = {
    'max_depth':5,
    'n_estimators':50,
    'learning_rate': 0.1,
    'seed': 0
}

run_model(xgb.XGBRegressor(**xgb_params), feats)

(-9569.227198767323, 72.83561801421891)

In [17]:
len(feats)

20

##Hyperopt

In [16]:
def obj_func(params):
  print("Training with params: ")
  print(params)
  
  # trenujemy model z użyciem cross walidacji
  mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)

  #  zwracamy słownik
  # status_ok - mowi o tym, ze udalo sie wyliczyc dana iteracje
  return {'loss': np.abs(mean_mae), 'status': STATUS_OK}

# space - przestrzen gdzie definujemy dla optymalizatora (hyperopt) granice poszukiwania rozsądnych wartości
xgb_reg_params = {
    'learning_rate':    hp.choice('learning_rate',  np.arange(0.05, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth',      np.arange(5, 16, 1, dtype=int)),
    'subsample':        hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'objective':        'reg:squarederror',
    'n_estimators':     100,
    'seed':             0
}

#run
best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)

best

Training with params: 
{'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.15000000000000002, 'max_depth': 14, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.8}
Training with params: 
{'colsample_bytree': 0.9500000000000001, 'learning_rate': 0.15000000000000002, 'max_depth': 8, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9}
Training with params: 
{'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.25, 'max_depth': 5, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 1.0}
Training with params: 
{'colsample_bytree': 0.5, 'learning_rate': 0.3, 'max_depth': 15, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Training with params: 
{'colsample_bytree': 0.8500000000000001, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Training with params: 
{'colsample_bytree': 0.55, 'le

{'colsample_bytree': 0.6000000000000001,
 'learning_rate': 2,
 'max_depth': 9,
 'subsample': 0.8}

In [14]:
feats = [ 'param_napęd__cat','param_rok-produkcji','param_stan__cat','param_skrzynia-biegów__cat','param_faktura-vat__cat','param_moc','param_marka-pojazdu__cat','feature_kamera-cofania__cat','param_typ__cat','param_pojemność-skokowa','seller_name__cat','feature_wspomaganie-kierownicy__cat','param_model-pojazdu__cat','param_wersja__cat','param_kod-silnika__cat','feature_system-start-stop__cat','feature_asystent-pasa-ruchu__cat','feature_czujniki-parkowania-przednie__cat','feature_łopatki-zmiany-biegów__cat','feature_regulowane-zawieszenie__cat' ]

xgb_params = {
    'learning_rate': 2,
    'max_depth':7,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'objective': 'reg:squarederror',
    'n_estimators':100,
    'seed': 0
}

run_model(xgb.XGBRegressor(**xgb_params), feats)

(-73685.96640784889, 430.9483180305935)

In [0]:
df = pd.read_hdf('data/car.h5')
df.shape
SUFFIX_CAT = '__cat'

for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values


df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]) )
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split('cm')[0].replace(' ','')) )

def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)


feats = ['param_napęd__cat',
   'param_rok-produkcji',
   'param_stan__cat',
   'param_skrzynia-biegów__cat',
   'param_faktura-vat__cat',
   'param_moc',
   'param_marka-pojazdu__cat',
   'feature_kamera-cofania__cat',
   'param_typ__cat',
   'param_pojemność-skokowa',
   'seller_name__cat',
   'feature_wspomaganie-kierownicy__cat',
   'param_model-pojazdu__cat',
   'param_wersja__cat',
   'param_kod-silnika__cat',
   'feature_system-start-stop__cat',
   'feature_asystent-pasa-ruchu__cat',
   'feature_czujniki-parkowania-przednie__cat',
   'feature_łopatki-zmiany-biegów__cat',
   'feature_regulowane-zawieszenie__cat']


In [19]:
def obj_func(params):
  print("Training with params: ")
  print(params)
  
  # trenujemy model z użyciem cross walidacji
  mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)

  #  zwracamy słownik
  # status_ok - mowi o tym, ze udalo sie wyliczyc dana iteracje
  return {'loss': np.abs(mean_mae), 'status': STATUS_OK}

# space - przestrzen gdzie definujemy dla optymalizatora (hyperopt) granice poszukiwania rozsądnych wartości
xgb_reg_params = {
    'learning_rate':        hp.choice('learning_rate', np.arange(0.05, 0.31, 0.05)),
    'max_depth':            hp.choice('max_depth',     np.arange(5, 16, 1, dtype=int)),
    'subsample':            hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bytree':     hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'objective':            'reg:squarederror',
    'n_estimators':         100,
    'seed':                 0,
}

# quniform - rozkład jednostkowy (dla danych typu float) (np. jak mam od 1 do 10 to prawdopodobienstwo że wystąpi któraś liczba z tego przedziału jest takie samo )
# choice iteruje z danego przedziału

# run
best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=25)

# algo - algorytm optymalizatora które uzywamy
# max_evals - liczba iteracji do wykonania
best

Training with params: 
{'colsample_bytree': 0.7000000000000001, 'learning_rate': 0.25, 'max_depth': 5, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.75}
Training with params: 
{'colsample_bytree': 0.75, 'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9}
Training with params: 
{'colsample_bytree': 0.9500000000000001, 'learning_rate': 0.2, 'max_depth': 11, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.55}
Training with params: 
{'colsample_bytree': 0.8500000000000001, 'learning_rate': 0.1, 'max_depth': 14, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 1.0}
Training with params: 
{'colsample_bytree': 0.8, 'learning_rate': 0.15000000000000002, 'max_depth': 12, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.75}
Training with params: 
{'colsample_bytree': 0.9, 'learning_rate': 0.05, 'max_dept

{'colsample_bytree': 0.8,
 'learning_rate': 1,
 'max_depth': 7,
 'subsample': 0.9500000000000001}